In [1]:
from docx import Document
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
from docx.table import Table
from docx.text.paragraph import Paragraph

def iter_block_items(parent):
    for child in parent.element.body:
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)

def get_table_paragraph_context_with_data(docx_path):
    doc = Document(docx_path)
    blocks = list(iter_block_items(doc))
    result = []
    last_para = None

    for block in blocks:
        if isinstance(block, Paragraph):
            if block.text.strip():
                last_para = block.text.strip()
        elif isinstance(block, Table):
            table = block
            rows = table.rows
            if not rows:
                continue  # ignore les tableaux vides

            # Nettoyage des headers
            headers = [cell.text.strip().lower().replace('\n', ' ') for cell in rows[0].cells]
            headers = [h for h in headers if h]

            champs = []
            for row in rows[1:]:
                champ = {headers[i]: cell.text.strip() for i, cell in enumerate(row.cells) if i < len(headers)}
                champs.append(champ)

            result.append({
                "context_paragraph": last_para,
                "table_data": champs,
                "block_index": len(result)
            })
    return result


In [2]:
get_table_paragraph_context_with_data("Specification_Transaction_Financiere.docx")

[{'context_paragraph': 'TRX',
  'table_data': [{'ordre': '1',
    'champ': 'ID Transaction',
    'format attendu': 'TRX  + 8 chiffres',
    'type': 'Texte',
    'contraintes supplémentaires': 'Unique, commence par TRX',
    'description': 'Identifiant unique de la transaction',
    'longueur (caractères)': '11',
    'requis': 'Oui',
    'valeur par défaut': 'Généré automatiquement'},
   {'ordre': '2',
    'champ': 'Date Transaction',
    'format attendu': 'AAAA-MM-JJ',
    'type': 'Date',
    'contraintes supplémentaires': 'Format ISO 8601',
    'description': 'Date de la transaction',
    'longueur (caractères)': '10',
    'requis': 'Oui',
    'valeur par défaut': 'Date du jour'},
   {'ordre': '3',
    'champ': 'Montant',
    'format attendu': 'Nombre décimal',
    'type': 'Float',
    'contraintes supplémentaires': 'En TND, positif, 2 décimales',
    'description': 'Montant de la transaction',
    'longueur (caractères)': '10',
    'requis': 'Oui',
    'valeur par défaut': '0.00'},
 

In [ ]:
from together import Together
import os
import json
import time


# ✅ 1. Authentification API
client = Together(api_key="ta_clé ")

# ✅ 2. Nom du modèle
MODEL_NAME = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"

# ✅ 3. Prompt amélioré
def build_prompt_json_ready(block_json):
    block_index = block_json.get("block_index", -1)
    instruction = f"""
Tu es un assistant intelligent chargé d'analyser un bloc issu d’un document de spécifications.

Ce bloc contient :
- Un paragraphe d’introduction (`context_paragraph`) décrivant le contexte du tableau.
- Un tableau structuré (`table_data`) contenant des informations techniques sous forme de lignes, avec des champs tels que "champ", "format attendu", "contraintes supplémentaires", etc.
- Un identifiant de bloc (`block_index`).

---

🎯 **Ton objectif** :
Détecter un **préfixe attendu** pour ce bloc.  
UN PRÉFIXE EXISTE TOUJOURS, MÊME S’IL EST IMPLICITE.

Un préfixe est une suite de lettres ou chiffres (souvent 2 à 5 caractères) utilisée comme début standardisé d’un identifiant ou d’un champ. Il peut être :
- clairement mentionné (explicite)
- déduit du format ou d’un motif répété dans les champs (implicite)

Même si le préfixe n’est pas formulé directement dans les textes, tu dois en **déduire le plus probable**, basé sur les exemples donnés.

---

🚫 Tu ne dois JAMAIS répondre `null` : déduis toujours un préfixe probable à partir des indices visibles.

---

📌 **Où peut-on trouver un préfixe ?**
- Dans le `context_paragraph`, par exemple :
  - "Le préfixe attendu est TRX"
  - "Chaque identifiant commence par AZE"
  - Sous format d'un mot isolé par exemple "04T"
- Dans le `table_data`, par exemple dans les colonnes :
  - "Format attendu" : "CBE + 8 chiffres"
  - "Contraintes supplémentaires" : "doit commencer par CLT001"

  
---

✅ **Exemples valides de préfixes** :
- `CBE`
- `AZE`
- `CLT001`
- `PRD`

🧠 Exemple implicite :  
Si dans une  colonne  tu vois `"07803 + 15 chiffres"`, alors le préfixe attendu est `"07803"`.


---
❗Tu dois répondre UNIQUEMENT avec le JSON demandé, sans aucune explication ni phrase en dehors du bloc JSON.
🔒 La réponse doit être strictement :
- au format JSON valide
- sans texte avant ou après
- sans Markdown (` ```json ` etc.)

📤 **Format de réponse strictement attendu (JSON uniquement)** :
{{
  "block_index": {block_index},
  "prefixe_detecte": "..."  ← un préfixe extrait, ou `null`
}}

---

Voici le bloc à analyser :
{json.dumps(block_json, indent=2, ensure_ascii=False)}
"""
    return instruction.strip()

# ✅ 4. Fonction de détection via LLaMA
def detect_prefix_llama_direct(block):
    prompt = build_prompt_json_ready(block)
    response_text =""

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
        )
        response_text = response.choices[0].message.content.strip()

        print(f"{block['block_index']}:\n{response_text}")
        return json.loads(response_text)
    except Exception as e:
        return {
            "block_index": block.get("block_index", -1),
            "prefixe_detecte": None,
            "error": str(e),
            "raw_response":response_text

        }

# ✅ 5. Intégration avec get_table_paragraph_context_with_data

In [4]:
# ✅ 5. Intégration avec get_table_paragraph_context_with_data
def run_prefix_detection_on_doc(docx_path):
    blocks = get_table_paragraph_context_with_data(docx_path)

    results = []
    for block in blocks:
        result = detect_prefix_llama_direct(block)
        prefix = result.get("prefixe_detecte")

        # ⚙️ Format final simplifié pour la suite du pipeline
        results.append({
            "prefixe_detecte": prefix,
            "table_data": block.get("table_data", []),
            "block_index": block.get("block_index", -1)  # facultatif mais utile pour le suivi
        })

        time.sleep(3)

    return results

In [5]:
block_with_prefix= run_prefix_detection_on_doc("Specification_Transaction_Financiere.docx")
block_with_prefix

0:
{"block_index": 0, "prefixe_detecte": "TRX"}
1:
{"block_index": 1, "prefixe_detecte": "CLT"}


[{'prefixe_detecte': 'TRX',
  'table_data': [{'ordre': '1',
    'champ': 'ID Transaction',
    'format attendu': 'TRX  + 8 chiffres',
    'type': 'Texte',
    'contraintes supplémentaires': 'Unique, commence par TRX',
    'description': 'Identifiant unique de la transaction',
    'longueur (caractères)': '11',
    'requis': 'Oui',
    'valeur par défaut': 'Généré automatiquement'},
   {'ordre': '2',
    'champ': 'Date Transaction',
    'format attendu': 'AAAA-MM-JJ',
    'type': 'Date',
    'contraintes supplémentaires': 'Format ISO 8601',
    'description': 'Date de la transaction',
    'longueur (caractères)': '10',
    'requis': 'Oui',
    'valeur par défaut': 'Date du jour'},
   {'ordre': '3',
    'champ': 'Montant',
    'format attendu': 'Nombre décimal',
    'type': 'Float',
    'contraintes supplémentaires': 'En TND, positif, 2 décimales',
    'description': 'Montant de la transaction',
    'longueur (caractères)': '10',
    'requis': 'Oui',
    'valeur par défaut': '0.00'},
   

In [6]:
def extract_pcr_lines(txt_path):
    with open(txt_path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]


In [7]:

def match_pcr_lines_to_blocks_by_prefix(blocks_with_prefixes, txt_path):
    pcr_lines = extract_pcr_lines(txt_path)
    matched_lines = []

    for i, line in enumerate(pcr_lines):
        matched = False
        for block in blocks_with_prefixes:
            prefix = str(block.get("prefixe_detecte", "")).strip()
            if prefix and line.startswith(prefix):
                matched_lines.append({
                    "line_index": i,
                    "line": line,
                    "matched_block": block
                })
                matched = True
                break
        if not matched:
            #print(f"[DEBUG] Ligne sans bloc associé : '{line}'")  # Optionnel
            matched_lines.append({
                "line_index": i,
                "line": line,
                "matched_block": None
            })

    return matched_lines


In [8]:
blocks_with_prefixes = run_prefix_detection_on_doc("Specification_Transaction_Financiere.docx")

res=match_pcr_lines_to_blocks_by_prefix(blocks_with_prefixes,"pcr_transactionFinanciere.txt")
res


2:
{
  "block_index": 2,
  "prefixe_detecte": "ABC"
}


[{'line_index': 0,
  'line': 'TRX000000012025-07-161500.50Crédit    Validé',
  'matched_block': None},
 {'line_index': 1,
  'line': 'CLT000001Alice Dupontalice.dupont@example.com     +216 98 765 432',
  'matched_block': None},
 {'line_index': 2,
  'line': 'ABCA12345678901234567890123412345',
  'matched_block': {'prefixe_detecte': 'ABC',
   'table_data': [{'ordre': '1',
     'champ': 'Code Banque',
     'format attendu': 'ABC  + 4 lettres majuscules',
     'type': 'Texte',
     'contraintes supplémentaires': 'Code BIC ou SWIFT',
     'description': 'Identifiant de la banque',
     'longueur (caractères)': '7',
     'requis': 'Oui',
     'valeur par défaut': 'XXXXXXX'},
    {'ordre': '2',
     'champ': 'Num compte',
     'format attendu': '24 chiffres',
     'type': 'Texte',
     'contraintes supplémentaires': 'Norme IBAN',
     'description': 'Compte bancaire du client',
     'longueur (caractères)': '24',
     'requis': 'Oui',
     'valeur par défaut': '000000000000000000000000'}],
   

In [9]:
def build_conformity_prompt(line, matched_block):
    import json
    return f"""
Tu es un assistant intelligent et rigoureux chargé de vérifier la conformité d’une ligne issue d’un fichier de Transactions PCR (Plan de Contrôle de Référence) avec les spécifications fournies.

---

Informations disponibles :  
- Ligne PCR à analyser :  
\"{line}\"

- Spécifications du bloc associé (tableau) :  
{json.dumps(matched_block["table_data"], indent=2, ensure_ascii=False)}

---

Ta mission est de :

1. Vérifier que tous les champs listés dans les spécifications sont présents dans la ligne PCR.

2. Pour chaque champ, contrôler que la valeur extraite respecte strictement les spécifications (type, longueur, préfixe, contraintes, etc.).

3. Vérifier que l’ordre des champs dans la ligne PCR correspond exactement à l’ordre défini dans les spécifications.

4. Si l’ordre est incorrect, proposer l’ordre corrigé sous forme d’une liste ordonnée des noms des champs.

5. Proposer une suggestion de ligne PCR corrigée qui respecte à la fois l’ordre, les longueurs, et les contraintes.

---

Consignes importantes :

- Ta réponse doit être strictement un objet JSON au format suivant, sans aucun texte ni explication supplémentaire.

- Utilise les clés exactes suivantes :

  - "line" : la ligne PCR analysée (chaîne de caractères)

  - "conforme" : "oui" ou "non", selon la conformité globale de la ligne


  - "champs" : une liste d’objets, un par champ, contenant :  
    - "nom" : nom du champ (ex: "Code Partenaire")  
    - "valeur" : valeur extraite du champ dans la ligne PCR  
    - "conforme" : "oui" ou "non"  
    - "erreur" : description précise de l’erreur (ou null si conforme)  
    - "longueur_attendue" : nombre entier (longueur attendue du champ)

  - "ordre_champs" : objet avec :  
    - "conforme" : "oui" ou "non"  
    - "ordre_attendu" : liste ordonnée des noms des champs selon les spécifications  
    - "ordre_lu" : liste ordonnée des noms des champs tels qu’ils apparaissent dans la ligne PCR  
    - "suggestion_ordre_corrige" : liste ordonnée des noms des champs dans l’ordre corrigé

  - "ligne_corrigee" : chaîne de caractères correspondant à la ligne PCR corrigée

---

Sois précis, rigoureux et concis dans ta réponse JSON.

---

Voici la ligne à analyser et ses spécifications :

"""


In [ ]:
def verify_conformity_with_llm(blocks_with_prefixes, txt_path):
    from together import Together
    import json
    import time

    client = Together(api_key="ta clé")

    # Affiche les correspondances ligne-préfixe
    matched = match_pcr_lines_to_blocks_by_prefix(blocks_with_prefixes, txt_path)
    results = []

    for line_obj in matched:
        line = line_obj["line"]
        matched_block = line_obj.get("matched_block")

        if not matched_block:
            results.append({
                "line": line,
                "conforme": False,
                "erreurs": ["Aucun bloc associé à cette ligne."],
                "debug": {
                    #"prefixes_detectes": [b.get("prefixe_detecte") for b in run_prefix_detection_on_doc(docx_path)],
                    "ligne_originale": line,
                    "ligne_sans_espaces": line.strip().replace(" ", "")
                }
            })
            continue

        prompt = build_conformity_prompt(line, matched_block)

        try:
            response = client.chat.completions.create(
                model="mistralai/Mixtral-8x7B-Instruct-v0.1",
                messages=[{"role": "user", "content": prompt}]
            )
            response_text = response.choices[0].message.content.strip()

            try:
                result = json.loads(response_text)
            except json.JSONDecodeError:
                result = {
                    "line": line,
                    "conforme": False,
                    "erreurs": ["Réponse JSON invalide du LLM."],
                    "raw_response": response_text
                }
            results.append(result)

        except Exception as e:
            results.append({
                "line": line,
                "conforme": False,
                "erreurs": [f"Erreur LLM : {str(e)}"]
            })

        time.sleep(2)

    return results


In [12]:
blocks_with_prefixes = run_prefix_detection_on_doc("Specification_Transaction_Financiere.docx")

verify_conformity_with_llm(blocks_with_prefixes,"pcr_transactionFinanciere.txt")

0:
{"block_index": 0, "prefixe_detecte": "TRX"}
1:
{"block_index": 1, "prefixe_detecte": "CLT"}


[{'line': 'TRX000000012025-07-161500.50Crédit    Validé',
  'conforme': 'non',
  'champs': [{'nom': 'ID Transaction',
    'valeur': 'TRX00000001',
    'conforme': 'non',
    'erreur': 'Format ID Transaction non respecté - Chiffres insuffisants',
    'longueur_attendue': 11},
   {'nom': 'Date Transaction',
    'valeur': '2025-07-16',
    'conforme': 'oui',
    'erreur': None,
    'longueur_attendue': 10},
   {'nom': 'Montant',
    'valeur': '1500.50',
    'conforme': 'non',
    'erreur': 'Format Montant non respecté - Devise manquante',
    'longueur_attendue': 10},
   {'nom': 'Type Transaction',
    'valeur': 'Crédit',
    'conforme': 'oui',
    'erreur': None,
    'longueur_attendue': 6}],
  'ordre_champs': {'conforme': 'non',
   'ordre_attendu': ['ID Transaction',
    'Date Transaction',
    'Montant',
    'Type Transaction'],
   'ordre_lu': ['ID Transaction',
    'Date Transaction',
    'Type Transaction',
    'Montant'],
   'suggestion_ordre_corrige': ['ID Transaction',
    'Date T